## Testing what we can do with mewpy

https://mewpy.readthedocs.io/en/latest/main.html#:~:text=Metabolic%20Engineering%20Workbench%20in%20python,based%20models%20and%20simulation%20approaches.


https://mewpy.readthedocs.io/en/latest/simulation.html


In [ ]:
! pip install mewpy

In [3]:
# Load libraries
import numpy as np
from cobra.io import read_sbml_model, write_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite
from cobra.core.gene import GPR

In [42]:
infilename = '../../models/iMM904.xml'
print(f"Loading {infilename}")
model = read_sbml_model(infilename)

Loading ../../models/iMM904.xml


In [43]:
# build a phenotype simulator
from mewpy.simulation import get_simulator
simul = get_simulator(model)

In [6]:
simul.metabolites[:10]

['2dr5p_c',
 '2hb_c',
 '2hb_e',
 '2hhxdal_c',
 '2hp6mbq_m',
 '2hp6mp_m',
 '2hpmhmbq_m',
 '2hpmmbq_m',
 '2ippm_c',
 '2kmb_c']

In [7]:
simul.reactions[:10]


['CITtcp',
 '13BGH',
 '13BGHe',
 '13GS',
 '16GS',
 '23CAPPD',
 '2DDA7Ptm',
 '2DHPtm',
 '2DOXG6PP',
 '2HBO']

In [8]:
simul.compartments

{'c': 'cytosol',
 'e': 'extracellular space',
 'm': 'mitochondria',
 'x': 'peroxisome/glyoxysome',
 'r': 'endoplasmic reticulum',
 'v': 'vacuole',
 'g': 'golgi apparatus',
 'n': 'nucleus'}

In [24]:
# find reactions identified with a term containing 'biomass'
simul.find('biomass', find_in='m')

""


In [65]:
# find metabolites identified with a term containing 'o2'
simul.find('o2',find_in='m')

,name,compartment,formula
id,,,
co2_c,CO2 CO2,c,CO2
co2_e,CO2 CO2,e,CO2
co2_g,CO2 CO2,g,CO2
co2_m,CO2 CO2,m,CO2
co2_n,CO2 CO2,n,CO2
co2_v,CO2 CO2,v,CO2
co2_x,CO2 CO2,x,CO2
h2o2_c,Hydrogen peroxide,c,H2O2
h2o2_m,Hydrogen peroxide,m,H2O2


A simulator may also be loaded considering environmental conditions, that will be considered during phenotype simulations. In the next example, glucose consumption is limited to 10 mmol/gDW/h while oxygen is set to unlimited.



In [113]:
# environmental conditions
envcond1 = {'EX_glc__D_e': (-100000.0, 100000.0),
           'EX_o2_e':(-1000,1000)}


envcond2 = {'EX_glc__D_e': (-0.000001, 100000.0),
           'EX_o2_e':(-1000,1000)}

envcond3 = {'EX_glc__D_e': (-100000.0, 100000.0),
           'EX_o2_e':(-10000000,0.0001)}

In [114]:
simul1 = get_simulator(model,envcond=envcond1)
result1 = simul1.simulate()
result1

objective: 2244.6488764932446
Status: OPTIMAL
Constraints: OrderedDict([('EX_glc__D_e', (-100000.0, 100000.0)), ('EX_o2_e', (-1000, 1000))])
Method:SimulationMethod.FBA

In [115]:
simul2 = get_simulator(model,envcond=envcond2)
result2 = simul2.simulate()
result2

/home/trinebera/miniconda3/lib/python3.9/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


objective: 0.026629505994517272
Status: INFEASIBLE
Constraints: OrderedDict([('EX_glc__D_e', (-1e-06, 100000.0)), ('EX_o2_e', (-1000, 1000))])
Method:SimulationMethod.FBA

In [106]:
simul3= get_simulator(model,envcond=envcond3)
result3 = simul3.simulate()
result3

objective: 9806.493729368463
Status: OPTIMAL
Constraints: OrderedDict([('EX_glc__D_e', (-100000.0, 100000.0)), ('EX_o2_e', (-10000000, 0.0001))])
Method:SimulationMethod.FBA

All phenotype simulations will consider the imposed environmental conditions, and as such they only need to be set once. Also, these conditions do not persistently alter the model, which can be reused with a different simulator instance.



## Phenotype simulation

Phenotype simulations are also run using the simulator instance using the simulate method.

In [62]:
# FBA 
result1 = simul1.simulate()
result2 = simul2.simulate()
result3 = simul3.simulate()

# or 
# result1 = simul1.simulate(method='FBA')

In [63]:
result1

objective: 2244.6488764932515
Status: OPTIMAL
Constraints: OrderedDict([('EX_glc__D_e', (1.0, 100000.0)), ('EX_o2_e', (-1000, 10))])
Method:SimulationMethod.FBA

In [35]:
result1 #objective: 2244.648876493278
result2 #objective: 2244.648876493278

# objective: 0.9748814206733505
# Status: OPTIMAL
# Constraints: OrderedDict([('EX_glc__D_e', (-10.0, 100000.0)), ('EX_o2_e', (-1000, 1000))])
# Method:FBA

objective: 2244.648876493278
Status: OPTIMAL
Constraints: OrderedDict([('EX_glc__D_e', (0.0, 100000.0)), ('EX_o2_e', (-1000, 1000))])
Method:SimulationMethod.FBA

In [37]:
result3 # objective: 2244.648876493278

objective: 2244.648876493278
Status: OPTIMAL
Constraints: OrderedDict([('EX_glc__D_e', (-100000.0, 0.0)), ('EX_o2_e', (-1000, 1000))])
Method:SimulationMethod.FBA

Flux Balance Analysis (FBA) can be run without identifying any method, or by passing the ‘FBA’ as method parameter. Other phenotype simulation methods may also be run using one of the identifiers:

 - Flux Balance Analysis: method = 'FBA'
 - Parsimonious FBA:method = 'pFBA'
 - Minimization of Metabolic Adjustment:method = 'MOMA'
 - Linear MOMA: method = 'lMOMA'
 - Regulatory on/off minimization of metabolic flux: method = 'ROOM'

In [29]:
# pFBA
result = simul.simulate(method = 'pFBA')
result

objective: 926.9119117145706
Status: OPTIMAL
Constraints: OrderedDict([('EX_glc__D_e', (-10.0, 100000.0)), ('EX_o2_e', (-1000, 1000))])
Method:pFBA

## Reaction fluxes